## Notebook to create brain state files from EF1_ALPHA excel sleep scoring spreadsheet

In [3]:
import os 
import numpy as np
import pandas as pd

In [64]:
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA')
epoch_number = "A, E, I, M, Q, U, Y, AC, AG, AK, AO, AS, BA, BE, BI, BM, BQ, BU, BY, CC, CG, CK, CO, CS, CW"
cols = "B, F, J, N, R, V, Z, AD, AH, AL, AP, AT, BB, BF, BJ, BN, BR, BV, BZ,CD, CH, CL, CP, CT, CX"
cols_discard = "C, G, K, O, S, W, AA, AE, AI, AM, AQ, AU, BC, BG, BK, BO, BS, BW, CA, CE, CI, CM, CQ, CU, CY"
skiprows = 5
raw_excel = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols, header= 1)

In [65]:
#list of animal ids
column_names = list(raw_excel)
animal_ids = [anim_id[0:10] for anim_id in column_names]

In [74]:
#strip whitespace
animal_ids = [id.replace(" ", "") for id in animal_ids]
print(animal_ids)

['210422_D', '210423_D', '210705_C', '210706_C', '191125_A', '191126_A', '210705_B', '210706_B', '210705_A', '210706_A', '191216_A_1', '191217_A_1', '191104_B', '191107_A', '191108_A', '210422_B_1', '210423_B_1', '191216_C', '191217_C', '191216_B', '191217_B', '210429_C', '210430_C', '210705_D', '210706_D']


In [67]:
raw_epochs = pd.read_excel('grant_EEG_classification.xlsx', usecols = epoch_number, header = 1, skiprows = 6)
raw_epochs_letter = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols, header = 1, skiprows = 6)
raw_discard = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols_discard, header = 1, skiprows = 6)

In [68]:
two_files = []
for epoch in raw_epochs:
    start_epochs = raw_epochs[raw_epochs[epoch] == 0].index.tolist()
    two_files.append(start_epochs)

In [76]:
for file, epoch, animal_id, epoch_letter_col, discard_col in zip(two_files, raw_epochs, animal_ids, raw_epochs_letter, raw_discard):
    print(animal_id)
    if len(file) == 2:
        start_2 = file[1:2]
        start_2 = start_2[0]
        end_epoch = start_2 - 1
        epochs_letter = raw_epochs_letter[epoch_letter_col]
        drop_na = epochs_letter.dropna()
        length_col = len(drop_na) 
        epoch_column_raw = raw_epochs[epoch]
        part_1 = epoch_column_raw[0: end_epoch].astype(int)
        part_2 = epoch_column_raw[start_2: length_col].astype(int)
        epochs_letter_part_1 = epochs_letter[0:end_epoch]
        epochs_letter_part_2 = epochs_letter[start_2: length_col]
        discard_data = raw_discard[discard_col]
        discard_part_1 = discard_data[0: end_epoch]
        discard_part_2 = discard_data[start_2: length_col]
        part_1_dict = {'epoch_numbers': part_1, 'brain_state': epochs_letter_part_1,
                       'epoch_discard_numbers': discard_part_1}
        part_2_dict = {'epoch_numbers': part_2, 'brain_state': epochs_letter_part_2,
                      'epoch_discard_numbers': discard_part_2}
        df_1 = pd.DataFrame(data = part_1_dict)
        df_2 = pd.DataFrame(data = part_2_dict)
        os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
        df_1.to_csv(str(animal_id) +'_part_1.csv', index=True) 
        df_2.to_csv(str(animal_id) +'_part_2.csv', index=True)
    else:
        epochs_letter = raw_epochs_letter[epoch_letter_col]
        drop_na = epochs_letter.dropna()
        length_col = len(drop_na) 
        epoch_column_raw = raw_epochs[epoch]
        all_epochs = epoch_column_raw[0:length_col].astype(int)
        all_epochs_letter = epochs_letter[0:length_col]
        discard_data = raw_discard[discard_col]
        all_discard_epochs = discard_data[0:length_col] 
        data_dict = {'epoch_numbers': all_epochs, 'brain_state': all_epochs_letter,
                       'epoch_discard_numbers': all_discard_epochs}
        os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
        data_df = pd.DataFrame(data = data_dict)
        data_df.to_csv(str(animal_id) +'.csv', index=True)

210422_D
210423_D
210705_C
210706_C
191125_A
191126_A
210705_B
210706_B
210705_A
210706_A
191216_A_1
191217_A_1
191104_B
191107_A
191108_A
210422_B_1
210423_B_1
191216_C
191217_C
191216_B
191217_B
210429_C
210430_C
210705_D
210706_D


In [ ]:
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
test_check = pd.read_csv('191125_A_part_2.csv')
test_check

In [ ]:
wake_indices = test_check.loc[test_check['brain_state'] == 'W'].index.tolist()
rem_indices = test_check.loc[test_check['brain_state'] == 'R']
nonrem_indices = test_check.loc[test_check['brain_state'] == 'N']
discard_indices = test_check.loc[test_check['epoch_discard_numbers'] == 'E']

In [ ]:
wake_indices_list = wake_indices.index.tolist()
rem_indices_list = rem_indices.index.tolist()
nonrem_indices_list = nonrem_indices.index.tolist()
discard_indices_list = discard_indices.index.tolist()

In [ ]:
def non_match_elements(list_a, list_b):
    non_match = []
    for i in list_a:
        if i not in list_b:
            non_match.append(i)
    return non_match

new_list = non_match_elements(wake_indices_list, discard_indices_list)

## check that code works on individual animals

In [4]:
os.chdir('/home/melissa/PROJECT_DIRECTORIES/ef1_alpha_analysis')
%run preprocess.py

In [5]:
#paths 
recording_path = '/home/melissa/PREPROCESSING/EF1_ALPHA'
brain_state_path = '/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder'

In [14]:
os.chdir('/home/melissa/PROJECT_DIRECTORIES/ef1_alpha_analysis')
%run ef1_alpha_properties.py
%run preprocess.py
%run filter.py
%run power.py

In [15]:
recordings_2_channels

{'A': ['191125', '191126'],
 'A_1': ['191217'],
 'B': ['191217', '210705'],
 'B_1': ['210422', '210423'],
 'C': ['210705', '191217'],
 'D': ['210422', '210423', '210705']}

In [16]:
channels_dict

{'A': [2, 3, 7, 9],
 'A_1': [2, 3, 7, 8],
 'B': [18, 19, 23, 25],
 'B_1': [18, 20, 23, 25],
 'C': [34, 35, 39, 41],
 'D': [50, 51, 55, 57]}

In [17]:
#2 lists of letters - 1 for one recording and 1 for two recording
recordings_letters_1 = ['A', 'A_1', 'B', 'C', 'D']
recording_letters_2 = ['A', 'A_1', 'B', 'B_1', 'C', 'D']

In [18]:
 print (dir(ExtractBrainStateEF1ALPHA)) 

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'create_epoch_bins', 'epoch_length', 'get_epoch_indices', 'load_brain_state_file', 'load_npy_recordings', 'remove_E_epochs', 'sample_rate']


In [19]:
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/two_recording/Part_2')
part_2 = np.load('191217_part2_A_1.npy')

In [20]:
animal_id = '191125'
letter = 'A'
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
state_file_names = os.listdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
for file in state_file_names:
    if file.startswith(animal_id) and file.endswith(letter + '_part_1.csv'):
        part_1_brain_state = pd.read_csv(file)
    if file.startswith(animal_id) and file.endswith(letter + '_part_2.csv'):
        part_2_brain_state = pd.read_csv(file)
        

In [21]:
part_1_brain_state

,Unnamed: 0,epoch_numbers,brain_state,epoch_discard_numbers
0,0,0,W,NaN
1,1,1,W,NaN
2,2,2,W,NaN
3,3,3,W,NaN
4,4,4,W,NaN
...,...,...,...,...
1124,1124,1124,W,NaN
1125,1125,1125,W,NaN
1126,1126,1126,W,NaN
1127,1127,1127,W,NaN


In [22]:
brainstates = ['W', 'R', 'N']
for brainstate in brainstates:
    for headstage_letters in recording_letters_2:
        print(headstage_letters)
        for anim_id in recordings_2_channels[headstage_letters]:
            print(anim_id)
            preprocessing_steps = ExtractBrainStateEF1ALPHA(anim_id, recording_path, brain_state_path, 
                                                            headstage_letters, recording_number=2)
            print(recording_path)
            part_1, part_2 = preprocessing_steps.load_npy_recordings()
            part_1_br_state, part_2_br_state = preprocessing_steps.load_brain_state_file()
            br_state_indices = preprocessing_steps.remove_E_epochs(part_1_br_state, brainstate)
            epoch_bins_1 = preprocessing_steps.get_epoch_indices(br_state_indices)
            print(epoch_bins_1)
            epoch_1 = preprocessing_steps.create_epoch_bins(part_1_br_state, epoch_bins_1)
            print(epoch_1)

A
191125
/home/melissa/PREPROCESSING/EF1_ALPHA
0
17
17
487
487
544
544
636
636
654
654
661
661
751
751
837
837
1008
1008
1040
1040
1049
[[0, 15], [17, 485], [487, 542], [544, 634], [636, 652], [654, 657], [661, 749], [751, 835], [837, 1005], [1008, 1038], [1040, 1047], [1049, 1128]]
[0, 5000, 10000, 17000, 22000, 27000, 32000, 37000, 42000, 47000, 52000, 57000, 62000, 67000, 72000, 77000, 82000, 87000, 92000, 97000, 102000, 107000, 112000, 117000, 122000, 127000, 132000, 137000, 142000, 147000, 152000, 157000, 162000, 167000, 172000, 177000, 182000, 187000, 192000, 197000, 202000, 207000, 212000, 217000, 222000, 227000, 232000, 237000, 242000, 247000, 252000, 257000, 262000, 267000, 272000, 277000, 282000, 287000, 292000, 297000, 302000, 307000, 312000, 317000, 322000, 327000, 332000, 337000, 342000, 347000, 352000, 357000, 362000, 367000, 372000, 377000, 382000, 387000, 392000, 397000, 402000, 407000, 412000, 417000, 422000, 427000, 432000, 437000, 442000, 447000, 452000, 457000, 4620

0
1021
1021
1070
1070
1093
1093
1100
1100
1104
1104
1140
1140
1143
1143
1170
1170
1187
1187
1190
1190
1198
1198
1240
1240
1370
1370
1568
1568
1585
1585
1601
1601
1605
1605
1865
1865
1941
1941
2128
2128
2134
2134
2213
2213
2262
2262
2279
2279
2288
2288
2301
2301
2304
2304
2311
2311
2322
2322
2330
2330
2357
2357
2371
[[0, 1019], [1021, 1031], [1070, 1091], [1093, 1094], [1100, 1100], [1104, 1137], [1140, 1141], [1143, 1167], [1170, 1181], [1187, 1188], [1190, 1196], [1198, 1238], [1240, 1368], [1370, 1456], [1568, 1568], [1585, 1598], [1601, 1603], [1605, 1628], [1865, 1865], [1941, 2124], [2128, 2132], [2134, 2209], [2213, 2234], [2262, 2275], [2279, 2282], [2288, 2294], [2301, 2302], [2304, 2307], [2311, 2317], [2322, 2328], [2330, 2355], [2357, 2362], [2371, 2372]]
[0, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 105000, 110000, 115000, 120000, 125000, 130000, 135000, 140000, 145000, 150000,

0
298
298
300
300
1357
1357
1402
1402
1404
1404
1412
1412
1415
1415
1585
1585
1715
1715
1889
1889
1945
1945
1973
1973
1981
1981
2128
2128
2150
2150
2226
2226
2240
[[0, 296], [298, 298], [300, 1354], [1357, 1400], [1402, 1402], [1404, 1410], [1412, 1413], [1415, 1583], [1585, 1594], [1715, 1715], [1889, 1890], [1945, 1968], [1973, 1975], [1981, 2126], [2128, 2143], [2150, 2150], [2226, 2236], [2240, 2240]]
[0, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 105000, 110000, 115000, 120000, 125000, 130000, 135000, 140000, 145000, 150000, 155000, 160000, 165000, 170000, 175000, 180000, 185000, 190000, 195000, 200000, 205000, 210000, 215000, 220000, 225000, 230000, 235000, 240000, 245000, 250000, 255000, 260000, 265000, 270000, 275000, 280000, 285000, 290000, 295000, 300000, 305000, 310000, 315000, 320000, 325000, 330000, 335000, 340000, 345000, 350000, 355000, 360000, 365000, 370000, 375000, 380000,

IndexError: list index out of range

In [ ]:
for brainstate in brainstates:
    for headstage_letters in recording_letters_2:
        print(headstage_letters)
        for anim_id in recordings_2_channels[headstage_letters]:
            print(anim_id)
            preprocessing_steps = ExtractBrainStateEF1ALPHA(anim_id, recording_path, brain_state_path, 
                                                            headstage_letters, recording_number=2)
            part_1, part_2 = preprocessing_steps.load_npy_recordings()
            part_1_br_state, part_2_br_state = preprocessing_steps.load_brain_state_file()
            if part_1 and part_2 is not None:
                epoch_indices_1 = preprocessing_steps.remove_E_epochs(part_1_br_state, brain_state_letter=brainstate)
                epoch_indices_2 = preprocessing_steps.remove_E_epochs(part_2_br_state, brain_state_letter=brainstate)
                epoch_bins_1 = preprocessing_steps.get_epoch_indices(part_1_br_state, epoch_indices_1)
                epoch_bins_2 = preprocessing_steps.get_epoch_indices(part_2_br_state, epoch_indices_2)
                for column, channel in zip(part_1_br_state.T, channels_dict[headstage_letters]):
                    filter_steps = Filter(column, epoch_bins_1)
                    filtered_data_1 = filter_steps.butter_bandpass()
                    power_steps = PowerSpectrum(filtered_data_1)
                    power_array_part_1, frequency_array_part_1 = power_steps.average_psd()
                for column, channel in zip(part_2_br_state.T, channels_dict[headstage_letters]):
                    filter_steps = Filter(column, epoch_bins_2)
                    filtered_data_2 = filter_steps.butter_bandpass()
                    power_steps = PowerSpectrum(filtered_data_2)
                    power_array_part_2, frequency_array_part_2 = power_steps.average_psd()
                average_power_array = average_power_df(power_array_part_1, power_array_part_2)
                dict_data = {'Animal_ID': [anim_id]*len(average_power_array), 'Headstage':[headstage_letters]*len(average_power_array),
                            'Channel': [channel]*len(average_power_array), 'Brainstate': [brainstate]*len(average_power_array),
                            'Power': average_power_array, 'Frequency': frequency_array_part_1,}
                two_recording.append(pd.DataFrame(data=dict_data))
    merged_power_file = pd.conctenate(one_recording, axis = 0).drop_duplicates().reset_index(drop = True)
    os.chdir('/home/melissa/RESULTS/EF1_ALPHA/2_REC')
    merged_power_file.to_csv(str(brainstate) + '_2_rec.csv', index = True)

In [ ]:
one_recording = []
for letters_1 in recordings_letters_1:
    for anim_id in recordings_1_channels[letters_1]:
        testing_class = ExtractBrainStateEF1ALPHA(anim_id, recording_path, brain_state_path, letters_1, recording_number = 1)
        npy_recording = testing_class.load_npy_recordings()
        print(npy_recording)
        br_state_files = testing_class.load_brain_state_file()
        if br_state_files is not None:
            br_state_indices = testing_class.remove_E_epochs(br_state_files, brain_state_letter = 'W')
            epoch_indices = testing_class.get_epoch_indices(br_state_indices)
            epoch_bins = testing_class.create_epoch_bins(br_state_files, epoch_indices)
            if npy_recording is not None:
                for column, channel in zip(npy_recording.T, channels_dict[letters_1]):
                    filter_test = Filter(column, epoch_bins)
                    filtered_data = filter_test.butter_bandpass()
                    power_test = PowerSpectrum(filtered_data)
                    power_array, frequency_array = power_test.average_psd()
                    dict_data = {'Animal_ID': [anim_id]*len(power_array), 'Headstage': [letters_1]*len(power_array),
                            'Channel': [channel]*len(power_array), 'Power': power_array, 'Frequency': frequency_array,
                                'BrainState': ['W']*len(power_array)}
                    one_recording.append(pd.DataFrame(data = dict_data))

In [ ]:
merged_power_file = pd.concat(one_recording, axis=0).drop_duplicates().reset_index(drop=True)

In [ ]:
os.chdir('/home/melissa/RESULTS/EF1_ALPHA/2_REC')
#merged_power_file.to_csv('one_recording_wake.csv', index=True)